## Target- urgent withdrawn by nurse(within 24hr)
* shift_applications status == 'cancelled' + shift_application prevStatus == 'confirmed'


### IDs -> only look at shifts application, leave each shifts after
* shift_applications id
* shift_applications user_id
* shift_applications shift_id


### Numeric Field
* shifts rate
* shifts net_pay: rate - nursedash profit
* shift_applications distance
* shift break time
* user withdrawn times: 
    Problem: 
        1. We don't know in reality that you will withdraw your previous one.
        2. cannot be used to predict new user
* user applied times:
* prev SA/CW rate


## Time
### shift, shift app time
* S_Create (shift created) => SA_Create (application created) => CWTime (comfim withdrawn) => Start_Time (shift start)
* S_Create: shift create time
* SA_Create: shift application create time
* CWTime: shift withdrawn time, whole confirmed withdrawn population (include within 24hr and > 24hr)
* Start_Time: shift start time
##### calculated field, graph in hours
* S_Create2SA_Create: shift application create time - shift create time
* S_Create2Start_Time: shift start time - shift create time
        1. has negative values. delete maybe?
* SA_Create2Start_Time: shift start time - shift application create time
        1. has negative values. delete maybe?

### User
* user approvedAt:
* user createdAt:
#### Calculated field
* U_Create2U_approved: User approve time - User create time
* U_Create2now: now - User create time, in months(seconds/2629746)
* U_Approve2now: now - User approve time


### Categorical
* shifts role: position name + type
* facilities name = facilities short_name
* facilities areaId = facilities areaName
* withdrawnInfo_key, withdrawnInfo_value
* facilities segmentName: Senior Living = 1, Healthcare = 0
* Users enableNotifications:
* Users emailNotifications:
* Users appNotifications:
* Users allowedNotifications:


### ?
* facilities allowedQualification
* facilities createdAt
* facilities rates
* nurse shift withdrawn by admin, when policy start, how many withdrawn by admin
* shifts = unit
* role
* users updateAt ?
* users relationToFacility

# Connect to DB, Fetch data

In [37]:
import pandas as pd
import numpy as np
# specify connection to database
import psycopg2
connection = psycopg2.connect(
    host="nursedash-prod.cuzi2kducsnv.us-east-1.rds.amazonaws.com",
    database="nursedash",
    user="external_analyst",
    password="vWHYpF9CNtC9KWBG7sZ5JvX9")

### <font color = green> all time to chicago time, No withdrawn info

In [38]:
df = pd.read_sql_query("""

SELECT  sa.id, sa.user_id, sa.shift_id, f.id AS facility_id, sa."withdrawnInfo" -> 'initiator' as withdrawnInfo_value,
sa."status", sa."prevStatus", "s"."status" AS "s_status", s."facility_id", "s"."description" AS "shift_description",
"s"."assigned_nurse_id", s."net_pay", "s"."unit" AS "s_unit",s."type", sa."hasNurseCheckEvent",
"s"."qualifications" AS "s_qualifications", "s"."breakTime" AS "s_breakTime", sa."withdrawnInfo",
"f"."name" AS "f_name","f"."short_name" AS "f_short_name", f."segmentName", f."areaName", 
timezone('America/Chicago', t."currentCheckInTime") as t_checkin,
timezone('America/Chicago', s."createdAt") as s_create,
timezone('America/Chicago', sa."createdAt") as sa_create,
timezone('America/Chicago', u."approvedAt") as u_approve,
timezone('America/Chicago', u."createdAt") as u_create,
timezone('America/Chicago', sa."statusUpdatedAt") as sa_statusUpdate,
timezone('America/Chicago', timezone('UTC', s.start_time)) AS "Start_Time" 
FROM shifts s
Left JOIN "TimeChangeRequests" t ON t."shiftId" = s.id
INNER JOIN shift_applications sa ON s.id = sa.shift_id
INNER JOIN facilities f ON s.facility_id = f.id
INNER JOIN users u ON sa.user_id = u.id

""", con = connection)

In [39]:
df.columns

Index(['id', 'user_id', 'shift_id', 'facility_id', 'withdrawninfo_value',
       'status', 'prevStatus', 's_status', 'facility_id', 'shift_description',
       'assigned_nurse_id', 'net_pay', 's_unit', 'type', 'hasNurseCheckEvent',
       's_qualifications', 's_breakTime', 'withdrawnInfo', 'f_name',
       'f_short_name', 'segmentName', 'areaName', 't_checkin', 's_create',
       'sa_create', 'u_approve', 'u_create', 'sa_statusupdate', 'Start_Time'],
      dtype='object')

### <font color = green> these features was done in sql before

### U_create2U_approve, in hours

In [40]:
from datetime import datetime
df['U_create2U_approve'] = df.apply(lambda row: (row['u_approve'] - row['u_create']).total_seconds()/3600, 
                                    axis = 1)

### U_approve2now, in month

In [41]:
df['U_approve2now'] = df.apply(lambda row: (datetime.now() - row['u_approve']).total_seconds()/2629746, 
                               axis = 1)

### CW_Time2Start_Time

In [42]:
# CW_Time2Start_Time
df['CW_Time2Start_Time'] = df.apply(lambda row: (row['Start_Time'] - row['sa_statusupdate']).total_seconds()/3600 if str(row['Start_Time'])[:2] != '00' else 9999, 
                                         axis = 1)

### Checkin time 2Start_Time, In minutes

In [43]:
df['checkin_Time2Start_Time'] = df.apply(lambda row: (row['Start_Time'] - row['t_checkin']).total_seconds()/60 if str(row['Start_Time'])[:2] != '00' else 99, 
                                         axis = 1)
# # fill na as 0
# df['checkin_Time2Start_Time'] = df['checkin_Time2Start_Time'].fillna(0)

# Data Cleaning

In [44]:
# sort by time that shift was created
df = df.sort_values(by = 'sa_create').reset_index(drop = True)

In [45]:
df[df['status'] == 'confirmed']['s_status'].value_counts()

pending review      27747
completed           23552
pending clock        1935
confirmed            1399
opened                  7
pending check-in        2
Name: s_status, dtype: int64

In [46]:
# what are prevstatus column like when status = confirmed
df[df['status'] == 'confirmed']['prevStatus'].value_counts()

selected       50184
unavailable       18
applied            6
cancelled          2
Name: prevStatus, dtype: int64

In [47]:
# what are status column like when prevstatus = confirmed
df[df['prevStatus'] == 'confirmed']['status'].value_counts()

withdrawn    17080
cancelled     8436
applied          2
Name: status, dtype: int64

# Feature Engineering

### Target: urgent withdrawn as 1

In [48]:
def CW_by_nurse(row):
    if row['status']=='withdrawn' and row['prevStatus'] == 'confirmed':
        if row['withdrawninfo_value'] == 'nurse':
            if row['CW_Time2Start_Time'] <= 24:
                return 1
            else:
                return 0
        else:
            return 0
    else:
        return 0
    
df['target'] = df.apply (lambda row: CW_by_nurse(row), axis=1)

In [49]:
df['target'].value_counts()

0    192832
1      6027
Name: target, dtype: int64

### Reliability Score

In [50]:
test = df[['id', 'user_id', 'CW_Time2Start_Time', 'target',
           'status','prevStatus', 'withdrawninfo_value', 
           'checkin_Time2Start_Time', 's_status', 'Start_Time']]
test['count'] = np.ones(len(test))
# sort by start time and user id
test = test.sort_values(by = ['user_id', 'Start_Time'], ascending = True).reset_index(drop = True)

# test[['status', 'prevStatus', 's_status', 'count']].groupby(by = ['status', 's_status']).sum()

<ipython-input-50-c631e4250a8f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['count'] = np.ones(len(test))


In [51]:
# create realibilty score dictionary
uid_library = list(pd.unique(test['user_id']))
Rscore_dict = {}
for uid in uid_library:
    Rscore_dict.update({uid:40})
    
# create Rs list to join with dataframe
RS_list = []
# fill Rs list
for i, v in enumerate(test['user_id']):
    ## record RS before event happen
    RS_list.append(Rscore_dict[v])
    
    ## calculate next application's RS
    
    # if UCW
    if test['status'][i] == 'withdrawn' and test['prevStatus'][i] == 'confirmed':
        # ucw < 5
        if test['CW_Time2Start_Time'][i] < 5: 
            Rscore_dict[v] -= 40
        # 5 < ucw < 12
        elif test['CW_Time2Start_Time'][i] < 12:
            Rscore_dict[v] -= 20
        # 12 < ucw < 24
        elif test['CW_Time2Start_Time'][i] < 24:
            Rscore_dict[v] -= 10
        # 24 < ucw < 48
        elif test['CW_Time2Start_Time'][i] < 48:
            Rscore_dict[v] -= 7
        # ucw > 48
        else:
            Rscore_dict[v] -= 3
    # if not ucw -> check if check in on time/early/late    
    else:
        if test['checkin_Time2Start_Time'][i] > 2:
            Rscore_dict[v] += 10
        else:
            Rscore_dict[v] += 3
            
test['reliability_score'] = RS_list

In [52]:
test = test[['id', 'reliability_score']]

In [53]:
df = df.merge(test, left_on='id', right_on = 'id')

### find extreme UCW rate facility from Top 30

In the TOP 30 facilities, Ohio Living Breckenridge Village,The Weils and Park Manor of South Belt has the highest UCW rate.

Whileas, The Hallmark and Village of Meyerland has the lowest UCW rate

In [54]:
df['f_highrate'] = df.apply(lambda row:1 if row['f_name'] in ['Ohio Living Breckenridge Village',
                                                              'The Weils', 
                                                              'Park Manor of South Belt'] else 0, axis = 1)
df['f_lowrate'] = df.apply(lambda row:1 if row['f_name'] in ['The Hallmark', 
                                                              'Village of Meyerland'] else 0, axis = 1)

###  Count previsous shift application

In [55]:
# track by using dictionary
count_prev_SA = []
test = df[['user_id', 'sa_create']]

# create SA dictionary, set all value = 0 -> dramatically reduce computational cost
uid_library = list(pd.unique(test['user_id']))
sa_dict = {} 
for uid in uid_library:
    sa_dict.update({uid: 0}) 
    
for i, v in enumerate(test['user_id']):
    sa_dict[v] += 1
    count_prev_SA.append(sa_dict[v]-1)
 
df['count_prev_SA'] = count_prev_SA

###  Count previsous urgent withdrawns

In [56]:
# track by using dictionary
count_prev_CW = []
test = df[['user_id', 'count_prev_SA', 'sa_create', 'target']]

# create cw dictionary, set all value = 0 -> dramatically reduce computational cost
uid_library = list(pd.unique(test['user_id']))
cw_dict = {} 
for uid in uid_library:
    cw_dict.update({uid: 0}) 
    
# fill dictionary and fill cw count
for i, v in enumerate(test['user_id']):
    if test['target'][i] == 1:
        cw_dict[v] += 1
        count_prev_CW.append(cw_dict[v]-1)
    else:
        count_prev_CW.append(cw_dict[v])
        
df['count_prev_CW'] = count_prev_CW

### Count previsous urgent withdrawns/Count previsous shift applications

In [57]:
df['prev_CW/SA_rate'] = df['count_prev_CW']/df['count_prev_SA']

# fill nan with 0, happend bc 0/0, meaning rate = 0
df['prev_CW/SA_rate'] = df['prev_CW/SA_rate'].fillna(0)

### Previous withdrawn times previous apply

In [58]:
df['prev_CW x SA_rate'] = df['count_prev_CW']*df['count_prev_SA']

### 2 Type dummy: RN, LVN + LPN , Rest


In [59]:
df['type_RN'] = df.apply(lambda row:1 if row['type'] == 'RN' else 0, axis = 1)
df['type_LVN+LPN'] = df.apply(lambda row: 1 if row['type'] == 'LVN' or row['type'] == 'LPN' else 0, axis = 1)

In [60]:
df['type_STNA'] = df.apply(lambda row:1 if row['type'] == 'STNA' else 0, axis = 1)

In [61]:
df['type_CNA'] = df.apply(lambda row:1 if row['type'] == 'CNA' else 0, axis = 1)

### 1 SegementName dummy: Senior Living, Healthcare

In [62]:
df['segmentName'].value_counts()

Senior Living    173822
Healthcare        25585
Internal            312
Name: segmentName, dtype: int64

In [63]:
def create_segmentName_dummy(row):
    if row['segmentName']=='Senior Living':
        return 1
    else:
        return 0
    
df['segmentName_d'] = df.apply(lambda row: create_segmentName_dummy(row), axis=1)

### 5 Facility Area Name Dummy

In [64]:
# # Houston
# df['areaName_houston'] = df.apply(lambda row: 1 if row['areaName'] == 'Houston' else 0, axis = 1)

# # Northeast Ohio
# df['areaName_no'] = df.apply(lambda row: 1 if row['areaName'] == 'Northeast Ohio' else 0, axis = 1)

# # DFW
# df['areaName_dfw'] = df.apply(lambda row: 1 if row['areaName'] == 'DFW' else 0, axis = 1)

# # Austin
# df['areaName_austin'] = df.apply(lambda row: 1 if row['areaName'] == 'Austin' else 0, axis = 1)

# # San Antonio
# df['areaName_san'] = df.apply(lambda row: 1 if row['areaName'] == 'San Antonio' else 0, axis = 1)

In [65]:
df = pd.concat([df, pd.get_dummies(df[['areaName']])], axis=1)
df = df.drop("areaName_San Antonio",axis=1)

### keep only prevstatus or status = confirmed

In [66]:
df = df[df.apply(lambda row: (row['prevStatus'] == 'confirmed') or (row['status'] == 'confirmed'), axis = 1)].reset_index(drop = True)

In [67]:
# how many left
df.shape

(80834, 51)

### How many previsous urgent withdrawns in a month

### <font color = green>output feature dataframe

In [68]:
df.columns

Index(['id', 'user_id', 'shift_id', 'facility_id', 'withdrawninfo_value',
       'status', 'prevStatus', 's_status', 'facility_id', 'shift_description',
       'assigned_nurse_id', 'net_pay', 's_unit', 'type', 'hasNurseCheckEvent',
       's_qualifications', 's_breakTime', 'withdrawnInfo', 'f_name',
       'f_short_name', 'segmentName', 'areaName', 't_checkin', 's_create',
       'sa_create', 'u_approve', 'u_create', 'sa_statusupdate', 'Start_Time',
       'U_create2U_approve', 'U_approve2now', 'CW_Time2Start_Time',
       'checkin_Time2Start_Time', 'target', 'reliability_score', 'f_highrate',
       'f_lowrate', 'count_prev_SA', 'count_prev_CW', 'prev_CW/SA_rate',
       'prev_CW x SA_rate', 'type_RN', 'type_LVN+LPN', 'type_STNA', 'type_CNA',
       'segmentName_d', 'areaName_Austin', 'areaName_Cincinnati',
       'areaName_DFW', 'areaName_Houston', 'areaName_Northeast Ohio'],
      dtype='object')

In [69]:
feature_df = df[['id', 'user_id', 'shift_id', 'prev_CW/SA_rate', 'status', 'U_approve2now','prev_CW x SA_rate',
                 'type_RN', 'type_LVN+LPN', 'type_STNA','segmentName_d', 
                  'net_pay', 'target', 'sa_create', 'Start_Time', 'areaName_Austin', 'areaName_Cincinnati',
                 'areaName_DFW', 'areaName_Houston', 'areaName_Northeast Ohio',
                  'count_prev_SA', 'count_prev_CW','f_highrate','f_lowrate','type_CNA','reliability_score']]
feature_df.to_csv('model_data.csv')

In [70]:
feature_df.shape

(80834, 26)

In [71]:
df[['reliability_score']]

,reliability_score
0,43
1,58
2,40
3,43
4,61
...,...
80829,316
80830,-43
80831,-40
80832,146
